## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [ ]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [ ]:
#!pip install --upgrade pip

In [2]:
#Better
#!pip install requests BeautifulSoup4 fire


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [2]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [3]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag='h2')

In [4]:
res
#print(res)
for i in res[100:]:
    res.remove(i)
    #print(res)
names_infl = []
handle_infl = []
for r in res:
    split_data = r.split('.',maxsplit=1)[1].rsplit('(',maxsplit=1)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].split(')',maxsplit=1)[0]
    names_infl.append(name)
    handle_infl.append(handle)
    #print(f'{name}:{handle}'
#print(handle_infl)
#df_influencer_name = pd.DataFrame(names_infl)
#df_influencer_handle = pd.DataFrame(handle_infl)
#df_influencer_name.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/Topinfluencer_handle.csv', index = False)

#print(df_influencer_handle)

In [5]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [6]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = get(url).content
re_gov = get_elements(response, tag='blockquote')
names = []
handles = []
for r in re_gov:
    split_data = r.split('— ',maxsplit=1)[1].rsplit('(',maxsplit=1)
    #print(split_data)
    name = split_data[0].split(',')[0].strip()
    handle =  split_data[1].rsplit(')',maxsplit=1)[0]
    names.append(name)
    handles.append(handle)

nam_handle = f'{name}:{handle}'
print(handles)
#print(df_gvrn)
#res[:2]

['@EswatiniGovern1', '@MalawiGovt', '@hagegeingob', '@FinanceSC', '@PresidencyZA', '@mohzambia', '@edmnangagwa', '@MinSantedj', '@hawelti', '@StateHouseKenya', '@PaulKagame', '@M_Farmaajo', '@SouthSudanGov', '@SudanPMHamdok', '@TZSpokesperson', '@KagutaMuseveni', '@angola_Mirex', '@willynyamitwe', '@Cherif_MZ', '@Presidence_RDC', '@PresidentABO', '@PresidenceBenin', '@rochkaborepf', '@PresidenciaCV', '@AOuattara_PRCI', '@Presidency_GMB', '@NAkufoAddo', '@President_GN', '@USEmbalo', '@PresidenceMali', '@CheikhGhazouani', '@IssoufouMhm', '@MBuhari', '@Macky_Sall', '@PresidentBio', '@MSPS_Togo']


In [20]:
df_gvrn = pd.DataFrame(handles, columns=["Twitter Handles"])
df_gvrn.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_influencers_handles.csv', index = False, header=True)
#print(df_gvrn)


In [7]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import string

import matplotlib.dates as mdates
import seaborn as sns
#sns.set()


# to view all columns
#pd.set_option("display.max.columns", None)

In [8]:
#!pip install tweepy
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [9]:
consumer_key = "re8a7dLArl8y2Tic312dbRWxH"
consumer_secret = "DcARRXgLiCfFE0y1aBFF9WlfHtLzZw2TgOCEOkM31rdKG8rJD0"
access_token = "994261917253472256-bRMXtnLl19sVm2a3flZv6t2WaW9b1AG"
access_token_secret = "HMnwkCLauS0OLOVNi62Ay1K9OcVlEvgawcfqOSuIIHbJz"

In [10]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [31]:
#getting tweets for 100 influencers
tweets = []
tweetCount=1
for i in handle_infl:
    try:
        results=api.user_timeline(id=i, count=tweetCount)
    except tweepy.TweepError as e:
                continue  
    for tweet in results:
        tweets.append(tweet.text)
#print(tweets)

In [26]:
#to convert tweets to csv
influencer_tweets = pd.DataFrame(tweets, columns=["Influencer Tweets"] )
print(influencer_tweets)    

                                    Influencer Tweets
0   What an inspiring interchange. Way to go Luke!...
1                 WE ARE LIVE https://t.co/vB1hS6EmZj
2      La video yase-butcher! https://t.co/6ZBC9vIs2X
3   📌 [JOB OPPORTUNITY] We're hiring an Info Finde...
4   RT @LukwesaBurak: Please watch this if you hav...
5   RT @A4A_Internet: The stakes are too high to k...
6   RT @globalcompact: Where other people saw a pr...
7   Our SEO wizard team located at Vancouver is re...
8   RT @YvonneNdege: The economic hit on remittanc...
9   He taught me to be kind but honest, open minde...
10  RT @gpgSocDev: MEC for @gpgSocDev Nomathemba M...
11  Instead of hosting its popular markets, @Kamer...
12  In the last few days, tens of thousands of peo...
13  RT @CBSNews: As the coronavirus pandemic cripp...
14  As a continent, Africa will experience a major...
15  RT @African_Markets: Airtel makes its debut as...
16  RT @KenRoth: The UN envoy in Yemen calls for a...
17  https://t.co/EfS2EsKmZ4 

In [49]:
# getting followers for 100 influencers
# Calling the get_user function with our parameters
followers = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue 
    followers.append(results.followers_count)
    #print(results.followers_count)
#print(followers)




In [54]:
# followers dataframe
df_followers = pd.DataFrame(followers, columns = ["Number of followers"])
df_followers.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/infl_followers.csv', index = False)
print(df_followers.head())


   Number of followers
0                25701
1                31288
2               101276
3                68070
4                21963


In [ ]:
# getting no of likes for 100 influencers
likes = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    likes.append(results.favourites_count)
    #print(results.followers_count)
#print(likes)

df_likes = pd.DataFrame(likes, columns=["No of likes"])
df_likes.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\top_influencers_likes.csv', index = False, header=True)
#print(df_likes)

In [57]:
# getting no of following for 100 influencers
following = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    following.append(results.friends_count)

    
print(following)



[37, 3059, 2840, 4591, 5048, 2876, 179, 0, 1732, 366, 2927, 1819, 2138, 893, 294, 172, 2988, 132, 425, 659, 2257, 813, 749, 941, 3605, 15, 0, 1155, 89620, 220, 1643, 49690, 151, 2416, 653, 1923, 1979, 983, 0, 1905, 3123, 1345, 3706, 4615, 406, 1008, 1143, 61980, 1114, 757, 1487, 657, 391, 1940, 25646, 5472, 3166, 1545, 74081, 10830, 141, 128, 11, 3348, 480, 987, 561, 1641, 2204, 985, 186, 305, 0, 455, 1618, 269, 2988, 25601, 0, 65, 530362, 1999, 109, 1223, 3948, 411, 478, 2304, 652, 631, 14, 356, 325]


In [59]:

# followers dataframe
d_following = pd.DataFrame(following, columns = ["Number of following"])
d_following.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/following.csv', index = False)
print(d_following.head())

   Number of following
0                   37
1                 3059
2                 2840
3                 4591
4                 5048


In [32]:
#getting retweets
no_of_retweets = []
for x in handle_infl:
    try:
        tweets = tweepy.Cursor(api.user_timeline, id=x, lang="en").items()
        for tweet in tweets:
            no_of_retweets.append(tweet.retweet_count)
            #print(tweet.retweet_count)
    except tweepy.TweepError as e:
        continue
print(no_of_retweets)

#retweets dataframe
df_retweets = pd.DataFrame(no_of_retweets, columns=["No of retweets"])
df_retweets.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\finalJ_retweets.csv', index = False, header=True)
print(df_retweets)


[]
Empty DataFrame
Columns: [No of retweets]
Index: []


In [48]:
#getting no of tweets shared
no_tweets_shared = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    no_tweets_shared.append(results.statuses_count)
    
print(no_tweets_shared)

11185


### Popularity reach
### Reach Score
### Relevance Score

In [ ]:
#popularity reach


In [62]:
#Reach Score = followers - following
reach = pd.concat([df_followers,d_following], axis=1)
print(reach.head())
reach['reach_score']= reach["Number of followers"] - reach["Number of following"]
print(reach.head()) 


   Number of following  Number of followers
0                   37                25701
1                 3059                31288
2                 2840               101276
3                 4591                68070
4                 5048                21963
   Number of following  Number of followers  reach_score
0                   37                25701        25664
1                 3059                31288        28229
2                 2840               101276        98436
3                 4591                68070        63479
4                 5048                21963        16915


In [77]:
retweets=[]
for i in id_num:
    try:
        results = api.retweets(id=i,count=10)
    except tweepy.TweepError as e:
        continue
    #print(results)
    #retweets.append(results.text)
#print(retweets)
    
    

In [ ]:
#hash_tags
hash_tags = []
for i in handle_infl:
    try:
        results = api.get_user(id=i)
    except tweepy.TweepError as e:
        continue
    if hasattr(status, "entities"):
        entities = status.entities
        # get hashtags
        if "hashtags" in entities:
            for ent in entities["hashtags"]:
                if ent is not None:
                    if "text" in ent:
                        hashtag = ent["text"]
                        if hashtag is not None:
                            hashtags.append(hashtag)
    
    


for status in Cursor(auth_api.user_timeline, id=handle).items():
                tweet_count+= 1
                if hasattr(status, "entities"):
                    entities = status.entities
                # get hashtags
                if "hashtags" in entities:
                    for ent in entities["hashtags"]:
                        if ent is not None:
                            if "text" in ent:
                                hashtag = ent["text"]
                                if hashtag is not None:
                                    hashtags.append(hashtag)

In [ ]:
#getting follower for gov
followers = []
for i in name:
    results = api.get_user(id=i)
    followers.append(results.followers_count)
    #print(results.followers_count)
#print(followers)

# followers dataframe
#df_followers = pd.DataFrame(followers, columns = ["Number of followers"])
#df_followers.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_followers.csv', index = False)
#print(df_followers.head())

In [74]:
# The Twitter users who we want to get tweets from
name = handles
# Calling the get_user function with our parameters


#print(tweet.retweet_count)
#print(tweet.favorite_count)

#no of likes
likes = []
for i in name:
    results = api.get_user(id=i)
    likes.append(results.favourites_count)
    #print(results.followers_count)
#print(likes)
df_likes = pd.DataFrame(likes, columns=["Twitter Handles"])
df_likes.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_influencers_likes.csv', index = False, header=True)
#print(df_likes)







    Twitter Handles
0                61
1               915
2               268
3                12
4                60
5               163
6                65
7               587
8               731
9                61
10              616
11               22
12              348
13               43
14                1
15               76
16              447
17             6258
18              466
19               98
20               16
21               94
22              475
23               91
24                4
25              452
26              133
27                6
28             6068
29             1732
30                0
31                3
32                8
33              530
34                0
35                1


In [31]:
#no of retweets
name = handles
retweets = []
for i in name:
    results = api.user_timeline(id=i)
    retweets.
    #print(results)
    if hasattr(results, 'retweet_count'):
        retweets = results.retweet_count
        #if retweets is not None:
           # retweet_count.append(retweets)

print(retweets)


SyntaxError: unexpected EOF while parsing (<ipython-input-31-8e67371b035e>, line 11)

In [25]:
name = handles
following = []
for i in name:
    results = api.get_user(id=i)
    following.append(results.friends_count)
    
print(following)
        
df_ffl = pd.DataFrame(following, columns=["Number of following"])
df_ffl.to_csv (r'C:\Users\HP\Desktop\CV, P.Statement and others\10 Academy\gov_influencers_following.csv', index = False, header=True)
#print(df_ffl)


[82, 26, 55, 224, 14, 95, 116, 127, 434, 214, 181, 2, 463, 115, 32, 28, 312, 4655, 195, 125, 4, 66, 151, 885, 23, 27, 352, 29, 181, 1001, 9, 17, 26, 171, 0, 2]


In [73]:
reach_score = pd.concat([df_gvrn,df_ffl], axis=1)
#print(reach_score)
#reach_score.sort_order()

     Twitter Handles  Number of following
0   @EswatiniGovern1                   82
1        @MalawiGovt                   26
2       @hagegeingob                   55
3         @FinanceSC                  224
4      @PresidencyZA                   14
5         @mohzambia                   95
6       @edmnangagwa                  116
7        @MinSantedj                  127
8           @hawelti                  434
9   @StateHouseKenya                  214
10       @PaulKagame                  181
11       @M_Farmaajo                    2
12    @SouthSudanGov                  463
13    @SudanPMHamdok                  115
14   @TZSpokesperson                   32
15   @KagutaMuseveni                   28
16     @angola_Mirex                  312
17    @willynyamitwe                 4654
18        @Cherif_MZ                  195
19   @Presidence_RDC                  125
20     @PresidentABO                    4
21  @PresidenceBenin                   66
22     @rochkaborepf              

In [65]:
len(api.retweets(197493438))

In [127]:
# Creating the API object while passing in auth information
# The Twitter user who we want to get tweets from
name = handles
# Calling the get_user function with our parameters
followers = []
for i in name:
    results = api.get_user(id=i)
   # print(results)
    #followers.append(results.followers_count)
    #print(results.followers_count)
#print(followers)

# followers dataframe
#df_followers = pd.DataFrame(followers, columns = ["Number of followers"])
#df_followers.to_csv('C:/Users/HP/Desktop/CV, P.Statement and others/10 Academy/gov_followers.csv', index = False)
#print(df_followers.head())



## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [ ]:
%%bash 

# curl the page and save content to tmp_file
#url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
#curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done